In [12]:
from quantum_mcmc_routines import *

In [ ]:
# define the model
np.random.seed(6)# should always be in the same cell!  
n_spins = 4

## construct problem Hamiltonian ##
shape_of_J=(n_spins,n_spins)

# defining J matrix (mutual 1-1 interaction)
J =  np.round(np.random.randn(n_spins,n_spins), decimals=2)#np.random.uniform(low= -1, high= 1, size= (n_spins, n_spins) )
J = 0.5 * (J + J.transpose() )
# print("J before:"); print(J)
J= J - np.diag(np.diag(J))

# J=np.array([[0,1,0.5,-1],[1,0,0.3,0.5],[0.5,0.3,0,1],[-1,0.5,1,0]])
print("J after:", J)

# defining h
h = np.round(np.random.randn(n_spins), decimals=2)#np.random.uniform(low= -1, high = 1, size= (n_spins))
# h=np.array([0.5]*n_spins)
print("h is:", h)

# instantiate the model
model = IsingEnergyFunction(J, h)
# print(model.get_energy('0100'))
alpha = np.sqrt(n_spins) / np.sqrt( sum([J[i][j]**2 for i in range(n_spins) for j in range(i)]) + sum([h[j]**2 for j in range(n_spins)])  )
print("alpha: ", alpha)

In [14]:
import time

In [ ]:
begin=time.time()
N_hops=50
a,b,c=quantum_enhanced_mcmc(
    N_hops=N_hops,
    num_spins=n_spins,
    initial_state='1011',
    num_elems=16,
    model=model,
    alpha=alpha,
    return_last_n_states=10,
    return_additional_lists=True,
    temp=0.4,
)
end=time.time()
print("titme taken:",(end-begin))

In [ ]:
print(b); print(c)

In [20]:
def run_mcmc_different_chains(num_spins:int, 
N_hops:int,num_seperate_mcmc_chains:int ,model,temp:float, 
return_last_n_states:int, is_quantum_mcmc=False, alpha=None ):

    num_elems=2**(num_spins)
    dict_seperate_chains_states_distn_mcmc={}
    dict_seperate_chains_sprime_mcmc={}
    dict_seperate_chains_accepted_mcmc={}
    poss_states=list(range(0,num_elems))
    print(f"Whether running quantum mcmc: {is_quantum_mcmc}")
    for chain_num in tqdm(range(0,num_seperate_mcmc_chains)):
        init_state=np.random.choice(poss_states)
        poss_states.remove(init_state)# to ensure that each mcmc chain starts with a different initial state
        initial_state=f'{init_state:0{num_spins}b}'#f'{np.random.randint(0,num_elems):0{num_spins}b}'
        if is_quantum_mcmc:
            dict_states_mcmc, state_mcmc_after_trsn, state_mcmc_after_accept =quantum_enhanced_mcmc(N_hops, num_spins, 
                                                                                initial_state,
                                                                                num_elems,model, 
                                                                                alpha,return_last_n_states=return_last_n_states,
                                                                                return_additional_lists=True, 
                                                                                temp=temp)
        else:
            dict_states_mcmc, state_mcmc_after_trsn, state_mcmc_after_accept =classical_mcmc(N_hops, num_spins, 
                                                                                initial_state,
                                                                                num_elems,model, 
                                                                                return_last_n_states=return_last_n_states,
                                                                                return_additional_lists=True, 
                                                                                temp=temp)                                                                        
        # sorting states in descending order of values(# occurences in mcmc chains)  for keys(states) 
        dict_states_mcmc_sorted_desc=value_sorted_dict(dict_states_mcmc, reverse=True)#dict_states_mcmc# this is where I might have to change things a little bit
        #storing in a dict
        dict_seperate_chains_states_distn_mcmc[chain_num]=dict_states_mcmc_sorted_desc
        dict_seperate_chains_sprime_mcmc[chain_num]=state_mcmc_after_trsn
        dict_seperate_chains_accepted_mcmc[chain_num]=state_mcmc_after_accept
    return dict_seperate_chains_states_distn_mcmc, dict_seperate_chains_sprime_mcmc, dict_seperate_chains_accepted_mcmc


In [54]:
from basic_utils import *

class caln_from_samples:
    '''  
    A class to use list_of_samples for different caln. 
    
    Method given here (but not limited to) can
    be come in handy for calculations of interest.
    '''
    def __init__(self, list_of_samples_sampled:list, num_mcmc_steps:dict):
        self.list_samples=list_of_samples_sampled
        self.num_mcmc_steps=num_mcmc_steps
        self.num_spins=len(list_of_samples_sampled[0])
        # import states function from basic utils.py
        self.all_poss_samples=states(num_spins=len(list_of_samples_sampled[0]))
        self.dict_count=self.count_states_occurence(list_samples=self.list_samples)
        self.dict_distn=self.empirical_distn(list_samples=self.list_samples)

    def count_states_occurence(self,list_samples)->dict:
        ''' 
        Function to get dict of occurence count of sample
        '''
        dict_count=dict(zip(self.all_poss_samples,[0]*(len(self.all_poss_samples))))
        dict_count.update(dict(Counter(list_samples)))
        return dict_count
    
    def empirical_distn(self, list_samples)->dict:
        ''' 
        Function to get dict of empirical distn from list of samples M.Chain was in.
        '''
        dict_distn=dict(zip(self.all_poss_samples,[0]*(len(self.all_poss_samples))))
        list_occurence_count=list(dict(Counter(list_samples)).values())
        normalised_values=list((1./(len(list_samples)))*np.array(list_occurence_count))
        dict_distn.update(dict( zip(list(dict_distn.keys()), normalised_values )))
        return dict_distn

    def running_avg_magnetization_as_list(self)->np.array:
        """
        Function to calculate the running average magnetization for the given mcmc trajectory as list
        
        Args:
        list_states_mcmc= List of state markov chain is in after each MCMC step
        
        Returns: array of running value of magnetization

        """
        list_of_strings = self.list_samples
        list_of_lists = (
            np.array([list(int(s) for s in bitstring) for bitstring in list_of_strings]) * 2
            - 1
        )
        return np.array(
            [
                np.mean(np.sum(list_of_lists, axis=1)[:ii])
                for ii in range(1, len(self.list_samples) + 1)
            ]
        )
    
    def average_of_some_observable(self,dict_observable_val_at_states: dict):
        return avg(dict_probabilities=self.dict_distn, dict_observable_val_at_states=dict_observable_val_at_states)

In [53]:
check=caln_from_samples(list_of_samples_sampled=c, num_mcmc_steps=N_hops)